In [2]:
pip install sgp4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 4.2 MB/s eta 0:00:00


In [1]:
from sgp4.api import Satrec,jday,days2mdhms
import numpy as np
from datetime import timedelta, datetime
from numba import jit, cuda
import csv
import pyproj

In [2]:
satellite_out=[]
px=[]
py=[]
pz=[]
vx=[]
vy=[]
vz=[]
LLA = []

In [1]:
def calc_time(year,month,day,hour,minute,second):
    '''function to compute a list of julian dates to pass to sgp4 function'''
    start = datetime(year,month,day,hour,minute,second)
    end = start + timedelta(hours=120)
    diff = timedelta(seconds=0.1)
    n = start
    jd = []
    fr = []
    while n <= end:
        n += diff
        jd_temp,fr_temp = jday(n.year,n.month,n.day,n.hour,n.minute,n.second)
        jd.append(jd_temp)
        fr.append(fr_temp)
    return(jd,fr)


In [4]:
def ecef2lla(i, pos_x, pos_y, pos_z):
    '''function to calculate latitude,longitude and altitude from given postion'''
    ecef = pyproj.Proj(proj="geocent", ellps="WGS84", datum="WGS84")
    lla = pyproj.Proj(proj="latlong", ellps="WGS84", datum="WGS84")
    lona, lata, alta = pyproj.transform(
    ecef, lla, pos_x[i], pos_y[i], pos_z[i], radians=False)
    return lona, lata, alta

In [5]:
with open('27000sats.txt') as f:
    lines = [line.rstrip() for line in f]

#@jit(target_backend='cuda')
def sat_details():
    print("computing satellite details ")
    for i in range(0,len(lines),2):
        s=lines[i]
        t=lines[i+1]
        satellite = Satrec.twoline2rv(s, t)
        month, day, hour, minute, second = days2mdhms(satellite.epochyr,satellite.epochdays)
        jd,fr = calc_time(2020,month,day,hour,minute,int(second))
        #k=0
        for j,f in zip(jd,fr):
            e, r, v = satellite.sgp4(j, f)
            #satellite_out.append([j+f,r,v])
            px.append(r[0])
            py.append(r[1])
            pz.append(r[2])
            vx.append(v[0])
            vy.append(v[1])
            vz.append(v[2])
            satellite_out.append([j+f,r[0],r[1],r[2],v[0],v[1],v[2]])
        #lona, lata, alta = ecef2lla(k, px, py, pz)
        print("######################################")

In [6]:
sat_details()

satellite1
######################################
satellite3
######################################
satellite5
######################################
satellite7
######################################
satellite9
######################################
satellite11
######################################
satellite13
######################################
satellite15
######################################
satellite17
######################################
satellite19
######################################
satellite21
######################################
satellite23
######################################
satellite25
######################################
satellite27
######################################
satellite29
######################################
satellite31
######################################
satellite33
######################################
satellite35
######################################
satellite37
######################################
satellite39
########################

In [7]:
print(satellite_out)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

